In [56]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')


## Read data and process data labels

In [57]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [58]:
data=read.csv('Quarterly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [59]:
dim(data)

[1] 24000    21

In [60]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,4,1,4,0.9492015,1.440272e-05,-3.801739,-1.3040882,-0.4829440,0.4354882,0.3478599,⋯,1,0.5299121,0.8423074,2.490487,-0.51403516,0.9484577,-0.7896943,1.7947057,0.5618988,25
2,4,1,4,0.9332456,2.024615e-05,-3.528016,-1.5740960,-0.3277471,0.3143940,0.1373249,⋯,1,0.3641703,0.8571066,2.440089,-0.29175138,0.2427210,-0.6669261,0.7095412,0.5378848,25
3,4,1,4,0.9753689,3.173153e-06,-4.202105,-1.3102058,-0.3300377,0.2486809,0.1689488,⋯,1,0.2498508,0.8961490,2.733696,-0.24714224,0.2873822,-0.7152289,0.8909252,0.6050959,25
4,4,1,4,0.9597504,7.178017e-06,-4.163362,-0.7087874,-0.2787477,0.4831910,0.1164472,⋯,1,0.4809287,0.8920654,2.726705,-0.20462399,0.2600985,-0.6048727,0.6906562,0.6227230,25
5,4,1,4,0.9770473,1.682990e-07,-5.666607,0.6009075,-0.5379779,0.5748836,0.9582223,⋯,1,0.5409435,0.5996603,2.068462,-0.14366107,2.4828596,-0.2911521,1.9555069,0.7981635,59
6,4,1,4,0.9884024,1.093815e-07,4.960507,-1.0952883,-0.1131941,0.3831215,0.6847558,⋯,1,0.5608638,0.8887061,2.856883,-0.07494233,0.7650798,-0.5923500,2.3289573,0.6350911,59


In [61]:
dlist= load('Quarterly_thetaf_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [62]:
dim(MASE)

[1] 24000     5     4

In [63]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [64]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [65]:
nanum

[1]   158   159   160   208   237   244   255   259   266   301   303   314
  [13]   322   327   330   340  1081  1201  1558  1564  1576  1579  1582  1585
  [25]  1586  1615  1626  1627  1629  1630  1639  1641  1644  1655  1656  1659
  [37]  1704  1709  1737  1744  1745  1751  1754  1756  1759  1763  1767  1769
  [49]  1773  1776  1778  1779  1784  1787  1788  1789  1792  1793  1794  1802
  [61]  1828  1855  1867  1881  1890  1898  1899  1902  1921  1933  1934  1935
  [73]  1937  1973  1974  2001  2023  2035  2053  2065  2071  2077  2091  2196
  [85]  2210  2217  2230  2233  2239  2283  2284  2300  2306  2323  2344  2399
  [97]  2436  2671  2676  2678  2703  2743  2781  3073  3149  3234  3235  3326
 [109]  3327  3328  3329  3330  3331  3332  3333  3334  3335  3336  3337  3448
 [121]  3449  3450  3451  3452  3453  3454  3455  3456  3514  3631  3634  3659
 [133]  3670  3672  3678  3700  3709  3747  3748  3749  3750  3751  3752  3753
 [145]  3754  3755  3756  3757  3758  3759  3760  3761  4121  4122  4123  4124
 [157]  4125  4126  4127  4128  4129  4130  4131  4132  4133  4134  4135  4136
 [169]  4158  4159  4160  4161  4162  4163  4164  4165  4166  4167  4168  4169
 [181]  4170  4171  4172  4173  4174  4175  4176  4177  4178  4179  4180  4181
 [193]  4182  4183  4184  4185  4186  4187  4188  4189  4190  4191  4192  4193
 [205]  4194  4195  4196  4197  4198  4199  4200  4201  4202  4203  4204  4411
 [217]  4412  4413  4415  4416  4417  4423  4442  4443  4444  4445  4446  4447
 [229]  4448  4449  4450  4451  4452  4453  4454  4455  4456  4457  4458  4459
 [241]  4460  4461  4462  4463  4464  4465  4466  4467  4468  4469  4470  4471
 [253]  4472  4473  4474  4475  4476  4477  4478  4519  4520  4523  4524  4525
 [265]  4526  4527  4528  4529  4530  4531  4532  4533  4534  4535  4536  4537
 [277]  4538  4539  4540  4541  4542  4543  4544  4545  4546  4547  4548  4549
 [289]  4550  4551  4552  4553  4554  4555  4556  4799  4813  4817  4823  4826
 [301]  4829  4831  4832  4836  4839  4840  4900  4924  4926  4946  4947  4952
 [313]  4953  4967  5032  5103  5104  5105  5117  5119  5120  5169  5178  5179
 [325]  5180  5181  5182  5183  5184  5185  5186  5187  5188  5189  5190  5307
 [337]  5308  5309  5514  5515  5516  5525  5528  5529  5530  5531  5532  5533
 [349]  5555  5559  5560  5576  5583  5584  5585  5589  5593  5594  5595  5596
 [361]  5600  5601  5602  5608  5617  5619  5631  5632  5634  5653  5656  5657
 [373]  5662  5664  5665  5679  5680  5684  5685  5686  5687  5688  5689  5691
 [385]  5703  5704  5713  5764  5770  5774  5781  5793  5795  5806  5807  5809
 [397]  5824  5825  5828  5842  5844  5854  5856  5871  5872  5886  5887  5901
 [409]  5911  5916  5917  5922  5924  5934  5935  5936  5937  5938  5939  6968
 [421]  6969  6970  6971  6972  6973  6974  6975  6976  6977  6978  6979  6980
 [433]  6981  6982  6983  6984  6985  6986  6987  6988  6989  6990  6991  6992
 [445]  6993  6994  6995  6996  6997  6998  6999  7000  7001  7002  7003  7004
 [457]  7005  7006  7007  7008  7021  7024  7026  7038  7039  7040  7041  7042
 [469]  7043  7044  7045  7046  7047  7048  7049  7050  7051  7052  7053  7054
 [481]  7055  7056  7057  7058  7059  7060  7061  7062  7063  7064  7065  7066
 [493]  7067  7068  7069  7070  7083  7101  7104  7110  7121  7122  7173  7175
 [505]  7197  7465  7466  7467  7468  7469  7470  7471  7472  7473  7474  7475
 [517]  7476  7477  7478  7479  7480  7481  7482  7483  7484  7485  7486  7487
 [529]  7488  7489  7490  7491  7492  7493  7494  7495  7496  7497  7498  7499
 [541]  7500  7501  7502  7503  7504  7505  7506  7507  7508  7509  7510  7511
 [553]  7512  7513  7514  7515  7516  7517  7518  7519  7520  7521  7522  7532
 [565]  7534  7536  7537  7538  7539  7540  7543  7544  7547  7550  7551  7552
 [577]  7555  7556  7557  7559  7564  7565  7567  7568  7569  7575  7578  7583
 [589]  7584  7585  7590  7591  7592  7593  7594  7595  7597  7600  7601  7602
 [601]  7605  7606  7607  7609  7631  7638  7639  764

In [66]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [67]:
table(realbestmin)

realbestmin
   0    1    2    3    4 
3023 2206 3357 5747 9667 

In [68]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [69]:
realbestmean

0
0
0
0
3
4
4
4
4
0
1


In [70]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,4,1,4,0.9492015,1.440272e-05,-3.801739,-1.3040882,-0.4829440,0.4354882,0.3478599,⋯,1,0.5299121,0.8423074,2.490487,-0.51403516,0.9484577,-0.7896943,1.7947057,0.5618988,25
2,4,1,4,0.9332456,2.024615e-05,-3.528016,-1.5740960,-0.3277471,0.3143940,0.1373249,⋯,1,0.3641703,0.8571066,2.440089,-0.29175138,0.2427210,-0.6669261,0.7095412,0.5378848,25
3,4,1,4,0.9753689,3.173153e-06,-4.202105,-1.3102058,-0.3300377,0.2486809,0.1689488,⋯,1,0.2498508,0.8961490,2.733696,-0.24714224,0.2873822,-0.7152289,0.8909252,0.6050959,25
4,4,1,4,0.9597504,7.178017e-06,-4.163362,-0.7087874,-0.2787477,0.4831910,0.1164472,⋯,1,0.4809287,0.8920654,2.726705,-0.20462399,0.2600985,-0.6048727,0.6906562,0.6227230,25
5,4,1,4,0.9770473,1.682990e-07,-5.666607,0.6009075,-0.5379779,0.5748836,0.9582223,⋯,1,0.5409435,0.5996603,2.068462,-0.14366107,2.4828596,-0.2911521,1.9555069,0.7981635,59
6,4,1,4,0.9884024,1.093815e-07,4.960507,-1.0952883,-0.1131941,0.3831215,0.6847558,⋯,1,0.5608638,0.8887061,2.856883,-0.07494233,0.7650798,-0.5923500,2.3289573,0.6350911,59


In [71]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [72]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [73]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,4,1,4,0.9492015,1.440272e-05,-3.801739,-1.3040882,-0.4829440,0.4354882,0.3478599,⋯,1,0.5299121,0.8423074,2.490487,-0.51403516,0.9484577,-0.7896943,1.7947057,0.5618988,25
2,4,1,4,0.9332456,2.024615e-05,-3.528016,-1.5740960,-0.3277471,0.3143940,0.1373249,⋯,1,0.3641703,0.8571066,2.440089,-0.29175138,0.2427210,-0.6669261,0.7095412,0.5378848,25
3,4,1,4,0.9753689,3.173153e-06,-4.202105,-1.3102058,-0.3300377,0.2486809,0.1689488,⋯,1,0.2498508,0.8961490,2.733696,-0.24714224,0.2873822,-0.7152289,0.8909252,0.6050959,25
4,4,1,4,0.9597504,7.178017e-06,-4.163362,-0.7087874,-0.2787477,0.4831910,0.1164472,⋯,1,0.4809287,0.8920654,2.726705,-0.20462399,0.2600985,-0.6048727,0.6906562,0.6227230,25
5,4,1,4,0.9770473,1.682990e-07,-5.666607,0.6009075,-0.5379779,0.5748836,0.9582223,⋯,1,0.5409435,0.5996603,2.068462,-0.14366107,2.4828596,-0.2911521,1.9555069,0.7981635,59
6,4,1,4,0.9884024,1.093815e-07,4.960507,-1.0952883,-0.1131941,0.3831215,0.6847558,⋯,1,0.5608638,0.8887061,2.856883,-0.07494233,0.7650798,-0.5923500,2.3289573,0.6350911,59


In [74]:
end_time = Sys.time()

In [75]:
time_matrix[1,]=end_time-start_time

## Target the interval where the actual error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [23]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [24]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:2.029481 
[2]	train-rmse:1.665235 
[3]	train-rmse:1.447392 
[4]	train-rmse:1.322725 
[5]	train-rmse:1.248085 
[6]	train-rmse:1.204537 
[7]	train-rmse:1.177559 
[8]	train-rmse:1.158221 
[9]	train-rmse:1.143578 
[10]	train-rmse:1.137660 
[11]	train-rmse:1.128342 
[12]	train-rmse:1.123690 
[13]	train-rmse:1.119953 
[14]	train-rmse:1.111438 
[15]	train-rmse:1.102567 
[16]	train-rmse:1.100391 
[17]	train-rmse:1.096965 
[18]	train-rmse:1.092910 
[19]	train-rmse:1.087410 
[20]	train-rmse:1.082003 
[21]	train-rmse:1.077437 
[22]	train-rmse:1.074060 
[23]	train-rmse:1.071030 
[24]	train-rmse:1.067517 
[25]	train-rmse:1.065224 
[26]	train-rmse:1.060955 
[27]	train-rmse:1.056242 
[28]	train-rmse:1.049534 
[29]	train-rmse:1.046125 
[30]	train-rmse:1.040433 
[31]	train-rmse:1.035510 
[32]	train-rmse:1.031480 
[33]	train-rmse:1.028297 
[34]	train-rmse:1.025062 
[35]	train-rmse:1.019171 
[36]	train-rmse:1.013150 
[37]	train-rmse:1.007714 
[38]	train-rmse:1.001704 
[39]	train-rmse:0.999

In [25]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.475339 
[2]	train-mlogloss:1.389121 
[3]	train-mlogloss:1.327478 
[4]	train-mlogloss:1.278605 
[5]	train-mlogloss:1.239705 
[6]	train-mlogloss:1.210149 
[7]	train-mlogloss:1.181129 
[8]	train-mlogloss:1.159859 
[9]	train-mlogloss:1.140331 
[10]	train-mlogloss:1.123300 
[11]	train-mlogloss:1.106456 
[12]	train-mlogloss:1.092783 
[13]	train-mlogloss:1.078794 
[14]	train-mlogloss:1.063188 
[15]	train-mlogloss:1.054987 
[16]	train-mlogloss:1.044774 
[17]	train-mlogloss:1.035447 
[18]	train-mlogloss:1.023158 
[19]	train-mlogloss:1.011710 
[20]	train-mlogloss:1.004577 
[21]	train-mlogloss:0.997120 
[22]	train-mlogloss:0.988839 
[23]	train-mlogloss:0.980306 
[24]	train-mlogloss:0.974533 
[25]	train-mlogloss:0.969884 
[26]	train-mlogloss:0.962534 
[27]	train-mlogloss:0.954474 
[28]	train-mlogloss:0.947018 
[29]	train-mlogloss:0.942797 
[30]	train-mlogloss:0.934816 
[31]	train-mlogloss:0.927642 
[32]	train-mlogloss:0.920668 
[33]	train-mlogloss:0.914359 
[34]	train-mlogloss

In [26]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048613 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4811
[LightGBM] [Info] Number of data points in the train set: 16927, number of used features: 21
[LightGBM] [Info] Start training from score 2.704673


In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4811
[LightGBM] [Info] Number of data points in the train set: 16927, number of used features: 21
[LightGBM] [Info] Start training from score -2.086021
[LightGBM] [Info] Start training from score -2.378472
[LightGBM] [Info] Start training from score -1.955944
[LightGBM] [Info] Start training from score -1.436882
[LightGBM] [Info] Start training from score -0.906268


In [28]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [76]:
start_time = Sys.time()

In [77]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [78]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [ ]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

In [ ]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

In [ ]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

In [ ]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

In [ ]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

## predict

In [37]:
start_time = Sys.time()

In [38]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [39]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [40]:
xgbregmin

[1]  2.218931198  0.286678851  0.396288693  0.072906435  3.491864681
    [6]  2.809981585  2.352921009  3.122053385  3.564653873  1.672953367
   [11]  0.904605389  2.136528254  2.553024530  2.543176651  1.729291558
   [16]  3.321441889  3.391129017  3.019443750  3.406294346  2.776618481
   [21]  3.286832333  3.564886808  2.679279089  2.004310369  2.735462666
   [26]  3.710011005  2.210883617  2.743405819  2.147833109  1.333508968
   [31]  2.678964376  2.053126812  1.400372982  2.279211998  1.750550866
   [36]  1.555159688  1.982086420  1.546833992  2.647464514  0.846171796
   [41]  1.719890475  1.769521832  2.012559652  1.208356619  0.906710148
   [46]  2.902548075  2.523645639  2.618072271  2.208630323  1.813696027
   [51]  2.188034058  1.930777550  2.101186275  1.663116932  1.577665091
   [56]  1.334786057  2.348031521  3.570945501  3.128428459  3.072715521
   [61]  3.818875313  2.492921829  2.406698942  4.049583435  2.424894333
   [66]  3.843268394  2.462169170  2.447923899  2.992512226  3.297719002
   [71]  2.640027285  2.019865274  2.921800137  2.935949326  2.551013947
   [76]  3.878059626  3.996287584  4.188398361  2.151210070  1.330011129
   [81]  0.566774905  1.831024647  0.558169782  3.424774170  3.900426865
   [86]  1.775673151  0.957506657  1.571263075  1.244917035  4.165319443
   [91]  1.014968157  0.512529969  2.058724880  3.594940662  3.985568047
   [96]  3.671413422  3.178812742  1.989837170  2.053798914  0.786625206
  [101]  0.669566393  2.227853775  0.696165621  3.931726456  3.233098030
  [106]  2.579125643  1.524642348  0.390324175  3.145015001  0.954612017
  [111]  0.378809124  2.485340357  0.995029628  1.125002265  1.381916165
  [116]  3.085433006  0.912533700  3.080402851  2.793235540  3.142437935
  [121]  3.131660461  2.791628361  3.873156786  3.224668026  3.348650455
  [126]  1.811024904  1.748117089  1.682963967  1.527040720  2.481143475
  [131]  3.210332155  2.694669724  3.442792416  1.727392912  2.244624853
  [136]  2.003801346  2.431325674  3.981285334  2.837845802  3.913667202
  [141]  2.876853704  3.931650639  4.143185139  3.870162249  4.022194386
  [146]  4.063960075  4.263317585  2.065068007  3.495791197  2.196158171
  [151]  2.468840122  0.326714993  2.551763773  3.166882753  2.937236309
  [156]  2.066295385  3.648551941  1.925151825  0.994973481  0.484702170
  [161]  1.626473904  0.388090014  2.417641401  1.018277049  1.451005578
  [166]  1.766864777  0.508539617  2.845718622  2.637085199  0.619354844
  [171]  3.372780323  1.648158073  2.098772526  1.467716813  2.427032471
  [176]  3.018446684  3.713165045  3.426224709  3.518457651  3.458604574
  [181]  3.193068027  1.101233721  1.466963053  0.305831730  0.344323635
  [186]  1.491218686  2.763760090  2.901021242  1.637857437  1.151490450
  [191]  1.618786693  0.178278998  0.231299400  1.286198378  0.357638538
  [196]  0.598192573  2.990544558  1.856517315  2.691463709  3.271293879
  [201]  2.305405617  3.265448332  2.390341520  2.477987766  3.485094547
  [206]  3.794033289  3.683138132  1.554982424  3.301856041  2.909014702
  [211]  3.313820601  2.448715210  3.031700850  2.482479572  1.687593460
  [216]  3.862113714  2.929364920  3.182556629  3.706932068  4.053162098
  [221]  3.229804039  3.971509695  3.773501873  2.946049929  3.455327511
  [226]  4.004029751  2.841140509  2.981519699  3.761317730  2.583822012
  [231]  3.754612684  0.868532360  3.907864809  3.221334934  3.468328714
  [236]  3.367230415  1.473568797  2.666738749  3.120538473  3.115236998
  [241]  2.681882620  2.970066547  1.528377652  3.016185522  3.575302601
  [246]  3.136823654  1.208634019  1.097469449  3.629689693  3.566660166
  [251]  2.914914370  1.849492788  3.884156227  3.151118755  1.674581409
  [256]  2.851497412  3.549462557  3.550083160  1.799100280  3.307065487
  [261]  2.912364244  2.834595442  3.225956440  3.204400778  2.738656759
  [266]  3.505355835  2.785525084  3.648149729  2.372972012  3.111104250
  [271]  2.565327883  2.157182455  4.126527786  2.715864420  2

In [41]:
xgbclsmin

[1] 3 0 0 0 4 4 4 4 4 0 0 4 4 3 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 2 0 4 1 4
   [37] 1 0 0 0 4 0 0 4 0 4 1 4 4 1 1 1 1 0 0 0 1 4 4 4 4 2 2 4 2 4 2 4 4 4 4 4
   [73] 4 4 4 4 4 4 4 0 0 4 0 4 4 0 4 0 0 4 4 0 4 3 4 4 3 2 4 0 0 3 0 4 4 3 4 0
  [109] 3 0 0 4 0 0 4 3 0 4 4 4 4 4 4 4 4 1 4 0 0 4 3 4 4 4 2 2 4 4 4 4 3 4 4 4
  [145] 4 4 4 2 4 1 4 0 1 3 4 4 4 0 1 0 0 0 4 0 0 0 0 4 4 0 4 0 0 0 0 4 4 4 4 4
  [181] 4 0 0 0 0 0 4 4 0 0 0 0 0 0 0 0 4 0 2 4 0 4 4 4 4 4 4 2 4 4 4 0 4 1 1 4
  [217] 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 1 4 4 4 3 1 4 4 2 2 4 0 3 4 4 1 0 4 4 4 4
  [253] 4 4 0 4 4 4 0 4 2 4 4 4 4 4 4 4 3 4 3 2 4 4 3 4 4 1 4 3 4 4 4 4 4 4 2 4
  [289] 2 2 4 4 3 4 3 4 0 4 2 4 2 4 3 2 0 0 0 2 0 1 4 4 0 0 0 2 2 4 4 2 4 1 2 2
  [325] 2 0 4 4 2 2 3 4 3 4 3 4 3 4 0 3 4 4 3 1 4 0 0 0 1 4 4 4 3 3 1 4 0 0 0 2
  [361] 2 1 0 1 4 4 4 0 4 2 0 4 2 0 4 4 2 3 2 4 4 0 2 4 0 0 4 0 1 4 1 1 0 0 4 4
  [397] 1 1 4 0 3 2 2 0 0 2 4 0 4 4 4 4 3 0 4 3 3 4 2 4 2 4 0 0 4 4 0 4 1 1 0 4
  [433] 4 0 4 1 4 4 4 4 4 2 1 2 4 0 3 4 3 4 1 4 4 4 4 0 4 4 4 3 0 2 4 2 1 4 4 1
  [469] 1 0 4 4 4 4 4 2 4 2 4 1 4 0 2 4 0 4 3 0 4 4 4 4 4 2 0 4 4 4 4 4 0 4 4 4
  [505] 4 4 0 0 0 4 0 4 4 1 4 0 0 0 1 4 4 4 4 4 4 4 4 0 0 4 4 2 4 0 4 4 1 1 4 2
  [541] 4 4 2 4 4 1 4 4 0 4 4 4 2 4 2 4 0 1 2 4 4 1 1 4 4 2 3 4 2 0 2 4 2 4 1 1
  [577] 1 1 3 4 0 4 2 2 4 4 4 4 4 2 4 4 3 4 4 4 0 4 1 4 4 4 1 4 3 2 2 4 4 4 0 4
  [613] 4 0 0 0 0 4 4 4 0 4 4 4 0 0 0 4 4 4 4 4 4 4 4 3 4 4 0 4 4 4 4 4 4 4 1 0
  [649] 3 4 4 4 4 4 4 4 4 1 2 0 3 0 4 0 0 2 4 4 4 0 1 3 4 1 4 1 4 3 4 4 4 1 0 2
  [685] 4 0 4 0 0 0 0 4 4 0 0 4 4 4 4 2 0 0 0 0 4 0 4 4 4 4 3 4 4 4 3 4 4 4 1 4
  [721] 4 4 4 1 1 4 4 4 4 4 0 4 1 4 0 1 1 4 4 4 4 4 4 0 4 4 4 2 4 4 1 4 4 1 1 4
  [757] 4 0 4 4 1 4 4 4 0 2 0 4 1 4 4 2 4 4 4 0 4 1 4 1 4 4 2 4 2 0 3 0 4 4 0 1
  [793] 4 2 3 0 4 3 3 3 4 4 3 2 0 0 0 4 4 0 1 4 2 4 2 1 4 3 0 4 4 2 4 4 4 3 4 4
  [829] 2 2 4 2 3 4 0 0 1 4 4 4 0 4 0 0 4 4 4 4 1 4 2 0 0 4 4 0 0 0 0 4 4 4 4 4
  [865] 4 4 0 4 4 0 1 0 0 0 4 4 0 4 4 4 4 4 3 4 4 4 4 4 1 4 4 4 3 0 3 0 3 0 4 4
  [901] 4 3 3 0 0 2 4 0 4 4 4 4 4 4 4 2 4 4 1 1 4 0 2 4 4 4 0 2 2 2 3 4 4 0 4 4
  [937] 4 4 1 4 4 4 0 4 2 0 1 2 2 4 4 4 4 0 4 1 1 4 4 0 4 2 0 4 0 4 0 4 2 2 0 4
  [973] 4 4 4 4 0 0 0 4 4 4 4 4 4 4 2 0 0 4 4 0 4 4 0 1 4 4 4 4 4 4 4 4 3 0 4 3
 [1009] 3 4 3 4 4 2 4 4 4 4 4 4 4 4 2 1 4 3 3 1 2 4 0 4 4 4 4 4 3 4 0 4 3 1 4 4
 [1045] 4 0 0 0 0 4 4 4 4 2 0 2 4 4 4 1 4 4 2 3 3 0 4 4 4 2 3 0 0 0 4 0 0 0 1 4
 [1081] 2 1 1 1 4 4 3 2 3 4 1 4 0 0 0 4 4 0 4 1 4 0 4 4 1 0 0 4 4 2 0 1 3 2 4 4
 [1117] 4 4 4 4 0 0 4 4 0 4 4 4 4 4 4 4 4 2 4 3 2 2 3 4 3 3 0 0 0 4 4 0 4 4 4 0
 [1153] 4 3 4 4 4 4 4 1 1 4 4 2 1 4 4 4 4 4 4 3 3 4 4 4 2 4 4 3 4 4 3 4 4 4 4 4
 [1189] 4 4 4 4 4 0 1 0 4 0 4 2 2 4 3 4 4 4 4 4 2 4 4 4 0 1 1 4 4 3 4 4 4 4 0 4
 [1225] 2 4 4 4 4 0 0 4 0 0 0 0 0 4 4 0 4 4 4 4 4 4 4 4 4 0 4 4 0 1 4 0 4 4 4 0
 [1261] 4 4 4 4 0 4 4 4 4 2 4 4 4 1 4 4 4 3 4 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 [1297] 0 3 4 4 0 0 2 4 2 4 4 4 4 4 0 0 3 2 4 4 4 4 4 4 4 0 0 4 4 4 4 0 3 3 4 1
 [1333] 4 0 2 4 2 4 4 4 3 4 4 4 2 0 4 2 0 1 4 2 3 2 4 4 4 4 3 2 2 2 4 4 4 0 4 4
 [1369] 4 4 3 2 4 4 4 4 2 4 4 4 4 4 4 4 4 4 2 4 4 2 4 4 1 4 4 1 1 0 0 1 1 4 4 0
 [1405] 4 4 2 4 0 2 4 1 0 0 2 2 2 3 4 4 4 4 3 3 2 0 3 3 2 4 4 4 0 0 0 0 4 1 4 4
 [1441] 2 3 0 4 4 4 4 4 2 1 4 4 4 2 1 2 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4
 [1477] 4 4 4 4 4 0 0 0 0 1 4 0 2 4 0 0 4 1 4 0 4 0 4 4 4 4 0 4 0 0 4 4 4 1 1 0
 [1513] 0 4 4 0 4 0 1 2 0 4 1 4 4 4 4 4 0 0 0 0 0 4 1 1 4 4 4 0 0 2 0 4 0 4 1 4
 [1549] 4 4 0 4 4 4 0 3 4 3 1 4 0 0 1 1 4 4 4 0 3 4 2 1 4 1 2 1 4 0 2 0 4 1 3 1
 [1585] 4 0 2 3 3 2 0 4 2 2 2 3 4 0 0 3 4 2 1 1 2 2 1 1 0 0 0 2 3 4 3 4 4 4 3 4
 [1621] 4 0 2 4 2 0 3 4 3 0 0 0 3 0 1 0 4 2 1 0 0 0 0 0 0 2 4 0 1 4 0 0 4 4 1 1
 [1657] 0 0 0 4 0 2 4 3 1 2 3 4 3 4 4 2 0 1 2 4 4 4 4 3 4 4 4 4 4 0 4 4 0 0 3 2
 [1693] 4 1 0 0 0 3 4 0 1 4 0 3 2 4 1 4 3 1 1 1 4 0 1 0 4 4 0 4 4 0 4 4 3 3 3 4
 [1729] 3 1 3 0 4 0 4 4 0 4 4 4 2 3 4 0 3 4 4 0 4 4 3 4 4 1 4 3 2 4 1 4 3 2 0 4
 [1765] 4 4 0 4 3 4 4 4 3 4 4 3 3 3 3 4 4 2 4 3 3 4 2 0 1 2 4 2 3 0 1 4 4 4 4 4
 [18

In [42]:
lgbregmin

[1]  1.46927627  0.85444737  0.05028974  1.21404956  2.46554788  2.62320578
    [7]  2.32857735  3.06057853  2.58197374  1.67491589  1.61801255  2.16595555
   [13]  2.65919321  2.67098148  2.35232154  2.93196462  2.99980580  2.77814438
   [19]  3.05310070  2.55374221  2.99954383  3.18366858  2.63726432  2.49382390
   [25]  2.40059030  3.47359816  2.35362795  2.69771501  2.31373302  1.86383308
   [31]  2.43670306  2.48021315  1.72277319  2.48389399  2.16387704  2.16430665
   [37]  2.28351367  2.03280683  2.41181688  1.19762051  1.76320561  2.29404807
   [43]  1.34209965  1.68853852  1.64552546  2.36634565  2.42799956  2.67587115
   [49]  2.06384071  2.15291399  2.43292022  2.26285340  2.28227232  2.11104017
   [55]  2.06095276  2.24472448  2.02978252  2.86291645  2.96224488  3.09688595
   [61]  3.72644494  2.57162549  2.73162625  3.72676991  2.09340075  3.61983634
   [67]  2.63924719  2.19741365  2.79468873  3.26675187  2.01677311  2.84035905
   [73]  3.05373198  2.84497454  2.55012870  3.33460923  4.01037220  4.01037220
   [79]  3.02730849  3.31175902  2.47896691  2.27196146  1.12772825  3.64098611
   [85]  3.92227190  1.12932657  1.41107340  1.22033170  1.03529512  3.69915027
   [91]  1.88472156  0.81420552  1.77208022  3.77378864  3.87309570  3.64556726
   [97]  3.30913728  2.02066021  2.37573576  1.56748014  1.39832252  1.46074744
  [103]  1.61157661  3.88301814  3.27411400  2.99616076  1.64553103  1.19988201
  [109]  3.58665604  1.54423438  1.33669828  2.41035445  1.42695928  1.34648643
  [115]  2.60557795  1.73342614  1.65011455  2.58395136  2.30909161  2.86315194
  [121]  2.33095428  2.14536326  3.50748466  2.33095428  2.41263990  1.66090941
  [127]  2.04843857  1.72872340  1.58657802  2.48478067  3.12322461  2.56329235
  [133]  2.94530171  2.40543095  2.18569384  1.81241052  2.80881767  3.90561505
  [139]  2.46917055  3.83260957  3.31082108  3.86158197  3.79189255  3.90991864
  [145]  3.95114346  3.67319128  3.81960182  2.35821032  3.12012991  2.03346299
  [151]  2.23592340  1.50338544  2.46039658  3.29843920  2.56432106  2.69765826
  [157]  3.47394747  1.75455985  1.02452163  1.21669781  1.92071679  1.15927487
  [163]  2.25502363  1.79560762  1.67978247  2.29128552  1.02702259  2.27642656
  [169]  2.50252604  0.73781859  3.00851929  1.46793485  1.80878227  1.42058258
  [175]  1.61971510  2.35245125  2.51973852  2.61323708  2.36599725  2.54962198
  [181]  2.32701200  0.96920794  1.63172308  1.16688243  1.28060685  1.93613628
  [187]  2.34199388  2.35005730  1.03402899  0.68833931  0.63824982  0.54300140
  [193]  0.82966609  0.72156959  0.77142685  0.41169746  1.03476712  1.66161685
  [199]  2.47156309  3.33372345  1.97899115  3.14366147  2.77398577  1.87102954
  [205]  3.55551817  3.32503481  3.79307984  1.72353769  3.16208796  2.89111289
  [211]  3.08952314  2.58808539  2.95964310  2.47404652  2.33475670  3.52845894
  [217]  3.32256800  3.13995490  3.39771170  3.68939632  3.11965331  3.74223919
  [223]  3.79213691  3.18026707  3.49665576  3.79269620  2.69734881  3.13311494
  [229]  3.26318368  2.56740943  3.57691604  1.26846781  3.58418300  2.44632731
  [235]  3.50991724  3.49074055  1.91187009  2.74803180  2.72906194  2.95190645
  [241]  2.65009407  2.98967582  1.35480051  2.70594055  3.39433886  2.86676872
  [247]  1.83577710  1.87019196  3.55695359  3.58574632  2.66538713  2.03129869
  [253]  3.85042872  3.28231140  2.17442859  2.69861553  3.42762850  3.48163270
  [259]  1.79422381  3.37652183  2.89674103  2.87239466  3.10228684  3.15151460
  [265]  2.89437978  2.91908804  2.61963505  3.61315409  2.95903573  2.66154567
  [271]  2.60102707  2.46973123  3.42640504  2.68837027  1.98813053  3.07749990
  [277]  3.02317242  1.73636316  3.01924863  2.75451874  2.95206368  3.46252213
  [283]  3.63751352  2.77332882  3.67053492  2.98967582  2.58468199  3.49535791
  [289]  2.59212589  2.68096744  3.52102537  2.02690390  2.65341394  3.05188539
  [295]  2.99021094  2.60769098  1.56678714  3.23584228  2.72357071  3.40748538
  [3

In [43]:
lgbclsmin

0.54456955,0.10422391,0.06614705,0.22086662,0.06419288
0.73411990,0.06371818,0.05812079,0.08335841,0.06068272
0.78573885,0.07862506,0.05516184,0.03959916,0.04087508
0.65574643,0.10368840,0.08598986,0.10599918,0.04857612
0.24973119,0.13203700,0.07706315,0.10354510,0.43762355
0.12100764,0.13028338,0.12097754,0.15260525,0.47512619
0.38269552,0.07277841,0.07777566,0.10369285,0.36305756
0.16288195,0.08182272,0.07384026,0.15123645,0.53021862
0.24115630,0.20399209,0.20640069,0.08959433,0.25885659
0.35305124,0.31258521,0.08771758,0.05348651,0.19315947
0.33441401,0.28303425,0.11666385,0.05890726,0.20698063


In [44]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [45]:
head(lgbclsminr)

2
1
4
2
3
4


In [46]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [47]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [48]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [49]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
3,2.21893120,2,1.46927627
0,0.28667885,1,0.85444737
0,0.39628869,4,0.05028974
0,0.07290643,2,1.21404956
4,3.49186468,3,2.46554788
4,2.80998158,4,2.62320578


In [50]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [51]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
0,0.2802930,2,0.5407501
0,0.8033683,4,0.5879173
0,-0.1292566,4,0.4604944
0,0.6234807,2,0.7524543
3,1.8427141,4,1.7924297
0,2.1479290,4,1.8828659


In [52]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
0,0.2802930,2,0.5407501
0,0.8033683,4,0.5879173
0,-0.1292566,4,0.4604944
0,0.6234807,2,0.7524543
3,1.8427141,4,1.7924297
0,2.1479290,4,1.8828659


In [53]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [54]:
result_list=list(preallmin,preallmean,time_matrix)

In [55]:
save(result_list, file = "Quarterly_thetaf_opt_pre_result.RData")

In [56]:
time_matrix

user_time,system_time,elapsed_time
3.7210128,3.7210128,3.7210128
2.7225070,2.7225070,2.7225070
2.4123126,2.4123126,2.4123126
0.8011491,0.8011491,0.8011491


## importance

In [84]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_cl)

# 打印特征重要性
print(importance_matrix)


              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:            length 0.128072054 0.099516010 0.06857921
 2:         curvature 0.119324309 0.089876667 0.08276620
 3:         linearity 0.114211859 0.084343178 0.07470422
 4:         seas_acf1 0.054883724 0.053164848 0.04941891
 5: seasonal_strength 0.052465646 0.064560410 0.06339650
 6:             spike 0.051381906 0.055805649 0.05324050
 7:            x_acf1 0.050106724 0.051965266 0.05057062
 8:            e_acf1 0.046201386 0.063968923 0.05748089
 9:           entropy 0.045611504 0.050985881 0.05643388
10:           e_acf10 0.044412857 0.049771149 0.06308240
11:             trend 0.044152213 0.055562756 0.04973301
12:           x_acf10 0.042506920 0.046776094 0.04486441
13:       diff1_acf10 0.041088231 0.054297192 0.05318815
14:         user_time 0.040334360 0.055088761 0.06344885
15:       diff2_acf10 0.038259457 0.046533450 0.04994241
16:        diff2_acf1 0.0356966

In [85]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_reg)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:         linearity 0.196458931 0.096019898 0.08404255
 2:         curvature 0.146041978 0.104770433 0.10404255
 3:            length 0.113992563 0.076301401 0.05127660
 4:         seas_acf1 0.061290721 0.064750766 0.04617021
 5: seasonal_strength 0.045064192 0.049052378 0.05255319
 6:            x_acf1 0.043617622 0.076646514 0.04829787
 7:           e_acf10 0.042457119 0.040979462 0.06212766
 8:           entropy 0.040941342 0.059430478 0.05468085
 9:            e_acf1 0.040903422 0.070180774 0.06340426
10:             spike 0.039992365 0.058862184 0.05510638
11:             trend 0.039444061 0.048451386 0.10382979
12:       diff1_acf10 0.033923470 0.055022216 0.04617021
13:         user_time 0.032999762 0.064505031 0.05617021
14:           x_acf10 0.032907029 0.035862458 0.03787234
15:       diff2_acf10 0.028519794 0.035857928 0.03978723
16:        diff2_acf1 0.0234909

In [86]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:         curvature 0.139986039 0.097955546 0.08497002
 2:         linearity 0.125953910 0.087677932 0.08037006
 3:            length 0.095826081 0.083919049 0.06388257
 4:         seas_acf1 0.057557865 0.056589297 0.05065123
 5:             spike 0.057074924 0.054243194 0.05649163
 6: seasonal_strength 0.053934728 0.060224040 0.06677693
 7:            x_acf1 0.053547852 0.060156182 0.05566467
 8:            e_acf1 0.048586570 0.065397015 0.06362415
 9:             trend 0.048441578 0.059346985 0.05643994
10:           entropy 0.048386329 0.068213226 0.06434774
11:       diff1_acf10 0.047194586 0.061161177 0.06124664
12:           e_acf10 0.045936106 0.066982744 0.06956791
13:           x_acf10 0.043057524 0.043897612 0.04491420
14:        diff2_acf1 0.040055887 0.038074412 0.04961753
15:        diff1_acf1 0.038866212 0.044275455 0.05065123
16:       diff2_acf10 0.0370587

In [87]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_mean_reg)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover   Frequency
               <char>       <num>       <num>       <num>
 1:         curvature 0.201965154 0.102858522 0.098312236
 2:         linearity 0.164172690 0.101060195 0.094936709
 3:            length 0.108270895 0.076101967 0.056751055
 4:         seas_acf1 0.062768438 0.058733962 0.046202532
 5:             spike 0.049822874 0.052504377 0.067299578
 6:           entropy 0.044368221 0.082747967 0.054219409
 7:            x_acf1 0.042106007 0.065348445 0.046413502
 8:           e_acf10 0.042059590 0.069658856 0.069831224
 9:             trend 0.040105011 0.055860322 0.111181435
10:            e_acf1 0.038958339 0.042603187 0.063291139
11: seasonal_strength 0.038402978 0.054598272 0.051265823
12:       diff1_acf10 0.034925182 0.046341666 0.047468354
13:       diff2_acf10 0.031978615 0.061084817 0.045991561
14:        diff1_acf1 0.030401740 0.038382993 0.042194093
15:           x_acf10 0.029568622 0.047128896 0.041139241
16:        dif

In [88]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_min_cl)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain      Cover  Frequency
               <char>       <num>      <num>      <num>
 1:            length 0.149349664 0.16664446 0.09793333
 2:         curvature 0.132531559 0.10592441 0.08726667
 3:         linearity 0.124740694 0.09829712 0.07000000
 4:         seas_acf1 0.056109391 0.05835925 0.05546667
 5:            x_acf1 0.050145486 0.05697163 0.05346667
 6: seasonal_strength 0.048461758 0.05474230 0.06186667
 7:             spike 0.046379519 0.05228306 0.05086667
 8:            e_acf1 0.044344545 0.05990684 0.05653333
 9:             trend 0.043422088 0.05143106 0.04740000
10:           entropy 0.040372613 0.03675922 0.05360000
11:           e_acf10 0.039937116 0.03447990 0.05820000
12:           x_acf10 0.037070561 0.04111186 0.04153333
13:       diff2_acf10 0.036585325 0.03660589 0.05046667
14:         user_time 0.036268037 0.03346625 0.05613333
15:       diff1_acf10 0.034573837 0.04477617 0.04813333
16:        diff1_acf1 0.030920329 0.03288587 0.0

In [89]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:         curvature 0.155627310 0.117906105 0.08833333
 2:         linearity 0.133834955 0.111967413 0.07720000
 3:            length 0.118657169 0.139081462 0.09560000
 4:         seas_acf1 0.059499721 0.060342243 0.05886667
 5:             spike 0.059365226 0.053676311 0.05980000
 6: seasonal_strength 0.049987485 0.045868077 0.06153333
 7:            x_acf1 0.049959402 0.063358841 0.05546667
 8:            e_acf1 0.046289297 0.056808353 0.06213333
 9:             trend 0.046078446 0.055617972 0.05306667
10:           entropy 0.044864328 0.046488280 0.06046667
11:           e_acf10 0.039866549 0.042102774 0.06020000
12:       diff1_acf10 0.039767810 0.043976834 0.05326667
13:           x_acf10 0.035955900 0.047681078 0.04320000
14:        diff1_acf1 0.034608532 0.035976439 0.04620000
15:        diff2_acf1 0.033856236 0.042361249 0.04920000
16:       diff2_acf10 0.0335222

In [90]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_reg)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain      Cover  Frequency
               <char>       <num>      <num>      <num>
 1:         curvature 0.246509578 0.11411650 0.10200000
 2:         linearity 0.192486324 0.13423666 0.09000000
 3:            length 0.143243833 0.19177850 0.11500000
 4:         seas_acf1 0.062706259 0.07319086 0.06500000
 5:            x_acf1 0.043833556 0.09124342 0.07000000
 6:             spike 0.037258163 0.05798660 0.05700000
 7:             trend 0.036174373 0.03858693 0.05600000
 8:            e_acf1 0.035377143 0.03311297 0.06300000
 9:           entropy 0.031217701 0.03991983 0.05433333
10: seasonal_strength 0.029634334 0.03849531 0.04833333
11:       diff1_acf10 0.024412101 0.03197465 0.04766667
12:           e_acf10 0.023696474 0.02990481 0.04733333
13:        diff2_acf1 0.022269619 0.02259088 0.04266667
14:        diff1_acf1 0.020307415 0.03415556 0.03900000
15:       diff2_acf10 0.019938457 0.01819790 0.03866667
16:           x_acf10 0.019599366 0.03232729 0.0

In [91]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'q_thetaf_imp_mean.csv')

In [92]:
importance

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.186022020,0.108209170,0.09340390
diff1_acf1,0.031045975,0.038197612,0.04451133
diff1_acf10,0.036574920,0.045863583,0.05241208
diff2_acf1,0.030211551,0.034654433,0.04460101
diff2_acf10,0.030624504,0.037983779,0.04575864
e_acf1,0.042302837,0.049480382,0.06301215
e_acf10,0.037889680,0.052162297,0.06173312
entropy,0.042209145,0.059342325,0.05834179
length,0.116499495,0.122720244,0.08280841


In [93]:
# 使用saveRDS保存模型到.rds文件
saveRDS(xgb_mean_reg, file = "Q_thetafxgbreg.rds")
saveRDS(xgb_mean_cl, file = "Q_thetafxgbcls.rds")
saveRDS(lgb_mean_reg, file = "Q_thetaflgbreg.rds")
saveRDS(lgb_mean_cl, file = "Q_thetaflgbcls.rds")